<a href="https://colab.research.google.com/github/Rossel/DataQuest_Courses/blob/master/034__Regular_Expression_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COURSE 5/6: DATA CLEANING IN PYTHON: ADVANCED

# MISSION 1: Regular Expression Basics

Learn to perform data cleaning with regular expressions.

## 1. Introduction

In the previous course, we learned that regular expressions are a powerful way of building patterns to match text. In the first two missions of this Data Cleaning Advanced course, we're going to extend our knowledge about this extremely powerful tool that every data scientist should be familiar with.

As powerful as regular expressions are, they can be difficult to learn at first and the syntax can look visually intimidating. As a result, a lot of students end up disliking regular expressions and try to avoid using them, instead opting to write more cumbersome code.

![IMG](https://s3.amazonaws.com/dq-content/354/difficult_regex_v2.svg)

That said, learning (and loving!) regular expressions is something that is a worthwhile investment

- Once you understand how they work, complex operations with string data can be written a lot quicker, which will save you time.
- Regular expressions are often faster to execute than their manual equivalents.
- Regular expressions are supported in almost every modern programming language, as well as other places like command line utilities and databases. Understanding regular expressions gives you a powerful tool that you can use wherever you work with data.

We could probably fill a whole Dataquest course with the intricacies of regular expressions, but instead we're going to give you a two-mission tour of the main components.

One thing to keep in mind before we start: don't expect to remember all of the regular expression syntax. The most important thing is to understand the core principles, what is possible, and where to look up the details. This will mean you can quickly jog your memory whenever you need regular expressions.

With that in mind, don't be put off if some things in these missions don't stick in your memory. As long as you are able to write and understand regular expressions with the help of documentation and/or other reference guides, you have all the skills you need to excel.



We'll be learning regular expressions while performing analysis on a dataset of submissions to popular technology site [Hacker News](https://news.ycombinator.com/).

![img](https://s3.amazonaws.com/dq-content/354/hacker_news.jpg)

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "stories") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles; stories that make it to the top of Hacker News' listings can get hundreds of thousands of visitors.





The dataset we will be working with is based off [this CSV of Hacker News stories from September 2015 to September 2016](). The columns in the dataset are explained below:

- `id`: The unique identifier from Hacker News for the story
- `title`: The title of the story
- `url`: The URL that the stories links to, if the story has a URL
- `num_points`: The number of points the story acquired, calculated as the total number of upvotes minus the total number of downvotes
- `num_comments`: The number of comments that were made on the story
- `author`: The username of the person who submitted the story
- `created_at`: The date and time at which the story was submitted


For teaching purposes, we have reduced the dataset from the almost 300,000 rows in its original form to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. You can download the modified dataset using the dataset preview tool.

Let's start by reading our Hacker News dataset into a pandas dataframe.

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Once you have completed verification, go to the CSV file in Google Drive, right-click on it and select “Get shareable link”, and cut out the unique id in the link.
# https://drive.google.com/file/d/1SgUoKVnxrer3-Yfvz4oBK0N9CzY6bJcu/view?usp=sharing
id = "1SgUoKVnxrer3-Yfvz4oBK0N9CzY6bJcu"

In [ ]:
# Download the dataset
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('hacker_news.csv')

In [ ]:
# import pandas library and read csv
import pandas as pd
hn = pd.read_csv("hacker_news.csv")

In [ ]:

# Get dataframe info
hn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            20099 non-null  int64 
 1   title         20099 non-null  object
 2   url           17659 non-null  object
 3   num_points    20099 non-null  int64 
 4   num_comments  20099 non-null  int64 
 5   author        20099 non-null  object
 6   created_at    20099 non-null  object
dtypes: int64(3), object(4)
memory usage: 1.1+ MB


In [ ]:
# Print dataframe dimensions
print(hn.shape)

(20099, 7)


**Instructions:**

1. Import the pandas library.
2. Read the `hacker_news.csv` file into a pandas dataframe. Assign the result to `hn`.
3. After you have completed the code exercise, use the variable inspector to familiarize yourself with the dataset.

In [ ]:
# Solution

import pandas as pd
hn = pd.read_csv("hacker_news.csv")

## 2. The Regular Expression Module

When working with regular expressions, we use the term pattern to describe a regular expression that we've written. If the **pattern** is found within the string we're searching, we say that it has **matched**.

As we previously learned, letters and numbers represent themselves in regular expressions. If we wanted to find the string `"and"` within another string, the regex pattern for that is simply `and`:

![img](https://s3.amazonaws.com/dq-content/354/basic_match_1.svg)

In the third example above, the pattern `and` does not match `Andrew` because even though `a` and `A` are the same letter, the two *characters* are unique.

We previously used regular expressions with pandas, but Python also has a built-in module for regular expressions: The `re` [module](https://docs.python.org/3/library/re.html#module-re). This module contains a number of different functions and classes for working with regular expressions. One of the most useful functions from the `re` module is the `re.search()` [function](https://docs.python.org/3/library/re.html#re.search), which takes two required arguments:

- The regex pattern
- The string we want to search that pattern for


In [ ]:
import re

m = re.search("and", "hand")
print(m)

<re.Match object; span=(1, 4), match='and'>


```
< _sre.SRE_Match object; span=(1, 4), match='and' >
```
The `re.search()` function will return a `Match` [object](https://docs.python.org/3/library/re.html#match-objects) if the pattern is found anywhere within the string. If the pattern is not found, `re.search()` returns `None`:

In [ ]:
m = re.search("and", "antidote")
print(m)

None


We'll learn more about match objects later. For now, we can use the fact that the boolean value of a match object is `True` while `None` is `False` to easily check whether our regex matches each string in a list. We'll create a list of three simple strings to use while learning these concepts:

In [ ]:
string_list = ["Julie's favorite color is Blue.",
               "Keli's favorite color is Green.",
               "Craig's favorite colors are blue and red."]

pattern = "Blue"

for s in string_list:
    if re.search(pattern, s):
        print("Match")
    else:
        print("No Match")

Match
No Match
No Match


So far, we haven't done anything with regular expressions that we couldn't do using the `in` keyword. The power of regular expressions comes when we use one of the special character sequences.

The first of these we'll learn is called a **set**. A set allows us to specify two or more characters that can match in a single character's position.



We define a set by placing the characters we want to match for in square brackets:

![img](https://s3.amazonaws.com/dq-content/354/set_syntax_breakdown.svg)

The regular expression above will match the strings `mend`, `send`, and `bend`.

Let's look at how we can add sets to match more of our example strings from earlier:

![img](https://s3.amazonaws.com/dq-content/354/basic_match_2.svg)


Let's take another look at the list of strings we used earlier:



In [ ]:
string_list = ["Julie's favorite color is Blue.",
               "Keli's favorite color is Green.",
               "Craig's favorite colors are blue and red."]

If you look closely, you'll notice the first string contains the substring `Blue` with a capital letter, where the third string contains the substring `blue` in all lowercase. We can use the set `[Bb]` for the first character so that we can match both variations, and then use that to count how many times `Blue` or `blue` occur in the list:



In [ ]:
blue_mentions = 0
pattern = "[Bb]lue"

for s in string_list:
    if re.search(pattern, s):
        blue_mentions += 1

print(blue_mentions)

2


We're going to use this technique to find out how many times Python is mentioned in the title of stories in our Hacker News dataset. We'll use a set to check for both `Python` with a capital 'P' and `python` with a lowercase 'p'.



**Instructions:**

We have provided code to import the `re` module and extract a **list**, `titles`, containing all the titles from our dataset.

1. Initialize a variable `python_mentions` with the integer value `0`.
2. Create a string — `pattern` — containing a regular expression pattern that uses a set to match `Python` or `python`.
3. Use a loop to iterate over each item in the `titles` list, and for each item:
 - Use the `re.search()` function to check whether `pattern` matches the title.
 - If `re.search()` returns a match object, increment (add `1` to) the `python_mentions` variable.

In [ ]:
# Code provided
titles = hn["title"].tolist()

In [ ]:
# Solution

python_mentions = 0
pattern = "[Pp]ython"

for t in titles:
    if re.search(pattern, t):
        python_mentions += 1

## 3. Counting Matches with pandas Methods

We've learned that we should avoid using loops in pandas, and that vectorized methods are often faster and require less code.

In the data cleaning course, we learned that the `Series.str.contains()` [method](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html) can be used to test whether a Series of strings match a particular regex pattern. Let's look at how we can replicate the example from the previous screen using pandas.

We'll start by creating a pandas object containing our strings:

In [ ]:
eg_list = ["Julie's favorite color is green.",
           "Keli's favorite color is Blue.",
           "Craig's favorite colors are blue and red."]

eg_series = pd.Series(eg_list)
print(eg_series)

0             Julie's favorite color is green.
1               Keli's favorite color is Blue.
2    Craig's favorite colors are blue and red.
dtype: object


Next, we'll create our regex pattern, and use `Series.str.contains()` to compare to each value in our series:

In [ ]:
pattern = "[Bb]lue"

pattern_contained = eg_series.str.contains(pattern)
print(pattern_contained)

0    False
1     True
2     True
dtype: bool


The result is a boolean mask: a series of `True`/`False` values.

One of the neat things about boolean masks is that you can use the `Series.sum()` [method]() to sum all the values in the boolean mask, with each `True` value counting as `1`, and each `False` as `0`. This means that we can easily count the number of values in the original series that matched our pattern:

In [ ]:
pattern_count = pattern_contained.sum()
print(pattern_count)

2


If we wanted, we could use method chaining to do the whole operation on one line:

In [ ]:
pattern_count = eg_series.str.contains(pattern).sum()
print(pattern_count)

2


Let's use this technique to replicate the analysis we did in the previous screen.

**Instructions:**

We have provided the regex pattern from the solution to the previous screen.

1. Assign the `title` column from the `hn` dataframe to the variable `titles`.
2. Use `Series.str.contains()` and `Series.sum()` with the provided regex pattern to count how many Hacker News titles contain `Python` or `python`. Assign the result to `python_mentions`.

In [ ]:
# Code provided
pattern = '[Pp]ython'

In [ ]:
# Solution

## 4. Using Regular Expressions to Select Data

On the previous two screens, we used regular expressions to count how many titles contain `Python` or `python`. What if we wanted to view those titles?

In that case, we can use the boolean array returned by `Series.str.contains()` to select just those rows from our series. Let's look at that in action, starting by creating the boolean array.

In [ ]:
titles = hn['title']

py_titles_bool = titles.str.contains("[Pp]ython")
print(py_titles_bool.head())

0    False
1    False
2    False
3    False
4    False
Name: title, dtype: bool


Then, we can use that boolean array to select just the matching rows:

In [ ]:
py_titles = titles[py_titles_bool]
print(py_titles.head())

102                  From Python to Lua: Why We Switched
103            Ubuntu 16.04 LTS to Ship Without Python 2
144    Create a GUI Application Using Qt and Python i...
196    How I Solved GCHQ's Xmas Card with Python and ...
436    Unikernel Power Comes to Java, Node.js, Go, an...
Name: title, dtype: object


We can also do it in a streamlined, single line of code:

In [ ]:
py_titles = titles[titles.str.contains("[Pp]ython")]
print(py_titles.head())

102                  From Python to Lua: Why We Switched
103            Ubuntu 16.04 LTS to Ship Without Python 2
144    Create a GUI Application Using Qt and Python i...
196    How I Solved GCHQ's Xmas Card with Python and ...
436    Unikernel Power Comes to Java, Node.js, Go, an...
Name: title, dtype: object


Let's use this technique to select all titles that mention the programming language Ruby, using a set to account for whether the word is capitalized or not.

**Instructions:**

Use `Series.str.contains()` to create a series of the values from `titles` that contain `Ruby` or `ruby`. Assign the result to `ruby_titles`.

In [ ]:
# Code provided
titles = hn['title']

In [ ]:
# Solution

## 5. Quantifiers

In the [data cleaning course](https://app.dataquest.io/course/python-datacleaning/), we learned that we could use braces (`{}`) to specify that a character repeats in our regular expression. For instance, if we wanted to write a pattern that matches the numbers in text from `1000` to `2999` we could write the regular expression below:
![img](https://s3.amazonaws.com/dq-content/354/quantifier_example.svg)

The name for this type of regular expression syntax is called a quantifier. Quantifiers specify how many of the previous character our pattern requires, which can help us when we want to match substrings of specific lengths. As an example, we might want to match both `e-mail` and `email`. To do this, we would want to specify to match `-` either zero or one times.

The specific type of quantifier we saw above is called a numeric quantifier. Here are the different types of numeric quantifiers we can use:

![img](https://s3.amazonaws.com/dq-content/354/quantifiers_numeric.svg)

You might notice that the last two examples above omit the first and last character as wildcards, in the same way that we can omit the first or last indicies when slicing lists.

In addition to numeric quantifiers, there are single characters in regex that specify some common quantifiers that you're likely to use. A summary of them is below.

![img](https://s3.amazonaws.com/dq-content/354/quantifiers_other.svg)

On this screen, we're going to find how many titles in our dataset mention `email` or `e-mail`. To do this, we'll need to use `?`, the optional quantifier, to specify that the dash character `-` is optional in our regular expression.

**Instructions:**

1. Use a regular expression and `Series.str.contains()` to create a boolean mask that matches items from `titles` containing `email` or `e-mail`. Assign the result to `email_bool`.
2. Use `email_bool` to count the number of titles that matched the regular expression. Assign the result to `email_count`.
3. Use `email_bool` to select only the items from `titles` that matched the regular expression. Assign the result to `email_titles`.

In [ ]:
# The `titles` variable is available from
# the previous screens

## 6. Character Classes

So far, we've learned how to perform simple matches with sets, and how to use quantifiers to specify when a character should repeat a certain number of times. Let's continue by looking at a more complex example.

Some stories submitted to Hacker News include a topic tag in brackets, like `[pdf]`. Here are a few examples of story titles with these tags:

```
[video] Google Self-Driving SUV Sideswipes Bus
New Directions in Cryptography by Diffie and Hellman (1976) [pdf]
Wallace and Gromit  The Great Train Chase (1993) [video]
```

In this screen, our task is going to be to find how many titles in our dataset have tags.

Our first inclination may be to create the regex `[pdf]`. Unfortunately, the brackets would be interpreted as a set, so our pattern would match the single characters `p`, `d`, or `f`.

![img](https://s3.amazonaws.com/dq-content/354/without_escaped_character_syntax_breakdown.svg)

To match the substring `"[pdf]"`, we can use backslashes to escape both the open and closing brackets: `\[pdf\]`.

![img](https://s3.amazonaws.com/dq-content/354/escaped_character_syntax_breakdown.svg)

The other critical part of our task of identifying how many titles have tags is knowing how to match the characters between the brackets (like `pdf` and `video`) without knowing ahead of time what the different topic tags will be.



To match unknown characters using regular expressions, we use **character classes**. Character classes allow us to match certain groups of characters. We've actually seen two examples of character classes already:

1. The set notation using brackets to match any of a number of characters.
2. The range notation, which we used to match ranges of digits (like `[0-9]`).

Let's look at a summary of syntax for some of the regex character classes:

![img](https://s3.amazonaws.com/dq-content/354/character_classes_v2_1.svg)


There are two new things we can observe from this table:

1. Ranges can be used for letters as well as numbers.
2. Sets and ranges can be combined.

Just like with quantifiers, there are some other common character classes which we'll use a lot.

![img](https://s3.amazonaws.com/dq-content/354/character_classes_v2_2.svg)


The one that we'll be using to match characters in tags is `\w`, which represents any number or letter. Each character class represents a single character, so to match multiple characters (e.g. words like `video` and `pdf`), we'll need to combine them with quantifiers.

In order to match word characters between our brackets, we can combine the word character class (`\w`) with the 'one or more' quantifier (`+`), giving us a combined pattern of `\w+`.

This will match sequences like `pdf`, `video`, `Python`, and `2018` but won't match a sequence containing a space or punctuation character like `PHP-DEV` or `XKCD Flowchart`. If we wanted to match those tags as well, we could use `.+`; however, in this case, we're just interested in single-word tags without special characters.



Let's quickly recap the concepts we learned in this screen:

- We can use a backslash to escape characters that have special meaning in regular expressions (e.g. `\` will match an open bracket character).
- Character classes let us match certain groups of characters (e.g. `\w` will match any word character).
- Character classes can be combined with quantifiers when we want to match different numbers of characters.

We'll use these concepts to count the number of titles that contain a tag.

**Instructions:**

1. Write a regular expression, assigning it as a string to the variable pattern. The regular expression should match, in order:
 - A single open bracket character.
 - One or more word characters.
 - A single close bracket character.
2. Use the regular expression to select only items from titles that match. Assign the result to the variable tag_titles.
3. Count how many matching titles there are. Assign the result to tag_count.

In [ ]:
# Answer provided for next chapters

pattern = "\[\w+\]"
tag_titles = titles[titles.str.contains(pattern)]
tag_count = tag_titles.shape[0]

## 7. Accessing the Matching Text with Capture Groups

On the previous screen, we learned that we can use backslashes to escape the `[` and `]` characters. Backslashes are used to escape many other characters in regular expressions, as well as to denote some special character sequences (like character classes).

In Python, a backslash followed by certain characters represents an escape sequence — like the `\n` sequence — which we previously learned represents a new line. These escape sequences can result in unintended consequences for our regular expressions. Let's take a look at a string containing the substring `\b`:

In [ ]:
print('hello\b world')

hello world


The escape sequence `\b` represents a backspace, so the final letter from our string is removed. The character sequence `\b` has a special meaning in regular expressions (which we'll learn about later), so we need a way to write these characters without triggering the escape sequence.

One way is to add an extra backslash before the "b":



In [ ]:
print('hello\\b world')

hello\b world


This can make regular expressions even more difficult to read and interpret, so instead we use [raw strings](https://docs.python.org/3/reference/lexical_analysis.html#string-and-bytes-literals), which we denote by prefixing our string with the `r` character. Let's take a look at the code from above with a raw string:

In [ ]:
print(r'hello\b world')

hello\b world


We strongly recommend using raw strings for every regex you write, rather than remember which sequences are escape sequences and using raw strings selectively. That way, you'll never encounter a situation where you forget or overlook something which causes your regex to break.

In the previous screen, we were able to calculate that 444 of the 20,100 Hacker News stories in our dataset contain tags. What if we wanted to find out what the text of these tags were, and how many of each are in the dataset?

In order to do this, we'll need to use **capture groups**. Capture groups allow us to specify one or more groups within our match that we can access separately. In this mission, we'll learn how to use one capture group per regular expression, but in the next mission we'll learn some more complex capture group patterns.

We specify capture groups using parentheses. Let's add an open and close parentheses to the pattern we wrote in the previous screen, and break down how each character in our regular expression works:

![img](https://s3.amazonaws.com/dq-content/354/tags_syntax_breakdown_v2.svg)

We'll learn how to access capture groups in pandas by looking at just the first five matching titles from the previous exercise:

In [ ]:
tag_5 = tag_titles.head()
print(tag_5)

66     Analysis of 114 propaganda sources from ISIS, ...
100    Munich Gunman Got Weapon from the Darknet [Ger...
159         File indexing and searching for Plan 9 [pdf]
162    Attack on Kunduz Trauma Centre, Afghanistan  I...
195               [Beta] Speedtest.net  HTML5 Speed Test
Name: title, dtype: object


In [ ]:
pattern = r"(\[\w+\])"
tag_5_matches = tag_5.str.extract(pattern)
print(tag_5_matches)

            0
66      [pdf]
100  [German]
159     [pdf]
162     [pdf]
195    [Beta]


We can move our parentheses inside the brackets to get just the text:

In [ ]:
pattern = r"\[(\w+)\]"
tag_5_matches = tag_5.str.extract(pattern)
print(tag_5_matches)

          0
66      pdf
100  German
159     pdf
162     pdf
195    Beta


If we then use `Series.value_counts()` we can quickly get a frequency table of the tags:

In [ ]:
tag_5_freq = tag_5_matches.value_counts()
print(tag_5_freq)

pdf       3
German    1
Beta      1
dtype: int64


Let's use this technique to extract all of the tags from the Hacker News titles and build a frequency table of those tags.

**Instructions:**

We have provided a commented line of code with the pattern from the previous exercise.

1. Uncomment the line of code and add parentheses to create a capture group *inside* the brackets.
2. Use `Series.str.extract()` and `Series.value_counts()` with the modified regex pattern to produce a frequency table of all the tags in the `titles` series. Assign the frequency table to `tag_freq`.

## 8. Negative Character Classes

On the previous screens, we wrote mostly simple regular expressions. In reality, regular expressions are often complex. When creating complex regular expressions, you often need to work iteratively so you can find "bad" instances that match your pattern and then exclude them.

In order to work faster as you build your regular expression, it can be helpful to create a function that returns the first few matching strings:

In [ ]:
def first_10_matches(pattern):
    """
    Return the first 10 story titles that match
    the provided regular expression
    """
    all_matches = titles[titles.str.contains(pattern)]
    first_10 = all_matches.head(10)
    return first_10

Another useful approach is to use an online tool like [RegExr](https://regexr.com/) that allows you to build regular expressions and includes syntax highlighting, instant matches, and regex syntax reference. For this screen, we'll use the `first_10_matches` function we just built to iteratively build a regular expression.

Earlier, we counted the titles that included Python — let's write a simple regular expression to match Java (another popular language), and use our function to look at the matches:

In [ ]:
first_10_matches(r"[Jj]ava")

267      Show HN: Hire JavaScript - Top JavaScript Talent
436     Unikernel Power Comes to Java, Node.js, Go, an...
580     Python integration for the Duktape Javascript ...
811     Ask HN: Are there any projects or compilers wh...
1023                         Pippo  Web framework in Java
1046    If you write JavaScript tools or libraries, bu...
1093    Rollup.js: A next-generation JavaScript module...
1162                 V8 JavaScript Engine: V8 Release 5.4
1195                   Proposed JavaScript Standard Style
1314           Show HN: Design by Contract for JavaScript
Name: title, dtype: object

We can see that there are a number of matches that contain `Java` as part of the word `JavaScript`. We want to exclude these titles from matching so we get an accurate count.

One way to do this is by using **negative character classes**. Negative character classes are character classes that match every character except a character class. Let's look at a table of the common negative character classes:
![img](https://s3.amazonaws.com/dq-content/354/negative_character_classes.svg)

Let's use the negative set `[^Ss]` to exclude instances like `JavaScript` and `Javascript`.

**Instructions:**

1. Write a regular expression that will match titles containing Java.
 - You might like to use the `first_10_matches()` function or a site like [RegExr](https://regexr.com/) to build your regular expression.
 - The regex should match whether or not the first character is capitalized.
 - The regex shouldn't match where 'Java' is followed by the letter 'S' or 's'.
2. Select every row from `titles` that match the regular expression. Assign the result to `java_titles`.

In [ ]:
# Provided code 
def first_10_matches(pattern):
    """
    Return the first 10 story titles that match
    the provided regular expression
    """
    all_matches = titles[titles.str.contains(pattern)]
    first_10 = all_matches.head(10)
    return first_10

In [ ]:
# Answer


## 9. Word Boundaries

On the previous screen, we used a negative set to find all of the mentions of "Java" in our dataset:

In [ ]:
first_10_matches(r"[Jj]ava[^Ss]")

436     Unikernel Power Comes to Java, Node.js, Go, an...
811     Ask HN: Are there any projects or compilers wh...
1840                    Adopting RxJava on the Airbnb App
1972          Node.js vs. Java: Which Is Faster for APIs?
2093                    Java EE and Microservices in 2016
2367    Code that is valid in both PHP and Java, and p...
2493    Ask HN: I've been a java dev for a couple of y...
2751                Eventsourcing for Java 0.4.0 released
2910                2016 JavaOne Intel Keynote  32mn Talk
3452    What are the Differences Between Java Platform...
Name: title, dtype: object

While the negative set was effective in removing any bad matches that mention JavaScript, it also had the side-effect of removing any titles where `Java` occurs at the end of the string, like this title:

```
Pippo  Web framework in Java
```

This is because the negative set `[^Ss]` must match one character. Instances at the end of a string aren't followed by any characters, so there is no match.

A different approach to take in cases like these is to use the **word boundary anchor**, specified using the syntax `\b`. A word boundary matches the position between a word character and a non-word character, or a word character and the start/end of a string. The diagram below shows all the word boundaries in an example string:

![img](https://s3.amazonaws.com/dq-content/354/word_boundaries.svg)

Let's look at how using a word boundary changes the match from the string in the example above:

In [ ]:
string = "Sometimes people confuse JavaScript with Java"
pattern_1 = r"Java[^S]"

m1 = re.search(pattern_1, string)
print(m1)

None


The regular expression returns `None`, because there is no substring that contains `Java` followed by a character that *isn't* `S`.

Let's instead use word boundaries in our regular expression:

In [ ]:
pattern_2 = r"\bJava\b"

m2 = re.search(pattern_2, string)
print(m2)

<re.Match object; span=(41, 45), match='Java'>


With the word boundary, our pattern matches the `Java` at the end of the string.

Let's use the word boundary anchor as part of our regular expression to select the titles that mention Java.



**Instructions:**

1. Write a regular expression that will match titles containing Java.
 - You might like to use the `first_10_matches()` function or a site like [RegExr](https://regexr.com/) to build your regular expression.
 - The regex should match whether or not the first character is capitalized.
 - The regex should match only where 'Java' is preceded and followed by a word boundary.
2. Select from `titles` only the items that match the regular expression. Assign the result to `java_titles`.

## 10. Matching at the Start and End of Strings

So far, we've used regular expressions to match substrings contained anywhere within text. There are often scenarios where we want to specifically match a pattern at the start and end of strings.

On the previous screen, we learned that the **word boundary anchor** matches the space between a word character and a non-word character. More generally in regular expressions, an **anchor** matches something that isn't a character, as opposed to character classes which match specific characters.

Other than the word boundary anchor, the other two most common anchors are the beginning anchor and the end anchor, which represent the start and the end of the string.

![img](https://s3.amazonaws.com/dq-content/354/positional_anchors.svg)

Note that the `^` character is used both as a beginning anchor and to indicate a negative set, depending on whether the character preceding it is a `[` or not.



Let's start with a few test cases that all contain the substring Red at different parts of the string, as well as a test function:

In [ ]:
test_cases = pd.Series([
    "Red Nose Day is a well-known fundraising event",
    "My favorite color is Red",
    "My Red Car was purchased three years ago"
])
print(test_cases)

0    Red Nose Day is a well-known fundraising event
1                          My favorite color is Red
2          My Red Car was purchased three years ago
dtype: object


If we want to match the word `Red` only if it occurs at the start of the string, we add the beginning anchor to the start of our regular expression:

In [ ]:
test_cases.str.contains(r"^Red")

0     True
1    False
2    False
dtype: bool

If we want to match the word `Red` only if it occurs at the end of the string, we add the end anchor to the end of our regular expression:

In [ ]:
test_cases.str.contains(r"Red$")

0    False
1     True
2    False
dtype: bool

Let's use the beginning and end anchors to count how many titles have tags at the start versus the end of the story title in our Hacker News dataset.

**Instructions:**

1. Count the number of times that a tag (e.g. `[pdf]` or `[video]`) occurs at the start of a title in `titles`. Assign the result to `beginning_count`.
2. Count the number of times that a tag (e.g. `[pdf]` or `[video]`) occurs at the end of a title in `titles`. Assign the result to `ending_count`.


## 11. Challenge: Using Flags to Modify Regex Patterns

Up until now, we've been using sets like [Pp] to match different capitalizations in our regular expressions. This strategy works well when there is only one character that has capitalization, but becomes cumbersome when we need to cater for multiple instances.

Within the titles, there are many different formatting styles used to represent the word "email." Here is a list of the variations:

```
email
Email
e Mail
e mail
E-mail
e-mail
eMail
E-Mail
EMAIL
emails
Emails
E-Mails
```

To write a regular expression for this, we would need to use a set for all five letters in email, which would make our regular expression very hard to read.

Instead, we can use **flags** to specify that our regular expression should ignore case.

Both `re.search()` and the pandas regular expression methods accept an optional `flags` argument. This argument accepts one or more flags, which are special variables in the re module that modify the behavior of the regex interpreter.



A [list of all available flags](https://docs.python.org/3/library/re.html#re.A) is in the documentation, but by far the most common and the most useful is the `re.IGNORECASE` [flag](https://docs.python.org/3/library/re.html#re.I), which is also available using the alias `re.I` for convenience.

When you use this flag, all uppercase letters will match their lowercase equivalents and vice versa. Let's look at an example without using the flag:

In [ ]:
email_tests = pd.Series(['email', 'Email', 'eMail', 'EMAIL'])
email_tests.str.contains(r"email")

0     True
1    False
2    False
3    False
dtype: bool

Now let's look at what happens when we use the flag:

In [ ]:
email_tests.str.contains(r"email",flags=re.I)

0    True
1    True
2    True
3    True
dtype: bool

No matter what the capitalization is, our regular expression matches.

We'll finish this mission by writing a regular expression and count the number of times that email is mentioned in story titles. You'll need to use both ignorecase as well as some of the other regex components you've already learned in this mission.

This screen is a challenge screen, so it's a little less guided than the exercises so far. As we mentioned at the start of this mission, regular expressions can be very complex, and unless you write them frequently, it's unlikely that you will remember all the syntax.



With that in mind, we don't expect that you will immediately remember how to perform this task so don't get disheartened if this exercise takes you more attempts than the other exercises in this mission. If you get stuck, you might try one or more of the following:

- Scanning over the regex concepts we've taught in the previous missions.
- Using the test cases that we'll provide.
- Using a web tool like [RegExr](https://regexr.com/) that lets you write a regex iteratively and see how it matches the test cases.


We've also provided a number of hints, however we strongly recommend trying to complete the challenge without them first. The skills you build as you try to solve the puzzle will be extremely valuable for you as you continue on your journey to becoming a data expert!

To help you test the regular expression that you build, we have provided a variable that includes each of the different ways "email" is included in the data.

**Instructions:**

1. Write a regular expression that will match all variations of email included in the starter code. Write your regular expression in a way that will be compatible with the ignorecase flag.
 - As you build your regular expression, you might like to use `Series.str.contains()` like we did in the examples earlier in this screen.
2. Once your regular expression matches all the test cases, use it to count the number of mentions of email in titles in the dataset. Assign the result to `email_mentions`.

In [ ]:
# Provided code
email_tests = pd.Series(['email', 'Email', 'e Mail', 'e mail', 'E-mail',
              'e-mail', 'eMail', 'E-Mail', 'EMAIL', 'emails', 'Emails',
              'E-Mails'])

In [ ]:
# Answer


## 12. Next Steps

In this mission, we learned the basics of using regular expressions to perform powerful text matching, including:
- Character classes to match certain groups of characters, including sets to match different capitalizations of programming languages.
- Quantifiers to match different quantities of characters, including matching different variations of "email."
- Negative character classes for matching anything except certain groups of characters.
- Word boundaries to match only specific instances of words.
- Positional anchors to match only at the start and end of strings.
- The ignorecase flag to make patterns case insensitive.

In the next mission, we'll expand on our regular expression knowledge with some advanced regex concepts!
